The the [day 9 puzzle](https://adventofcode.com/2021/day/9) involves analyzing a grid to discover "low points" in the grid and then extrapolate out "basins" from those low points.

     - A low point is when no adjacent numbers are lower than it. 
     - A basin is all points touching a low point (or parts of the basin) that increment up in value without hitting 9.

In [1]:
import gridthings
import pandas  # only for printing out the grid, not for analysis

In [2]:
data = """
2199943210
3987894921
9856789892
8767896789
9899965678
"""

# Use an IntGrid vice regular Grid to cast each value to int
#
# Setting the out_of_bounds_value to 10 so that when we
# compare whether one cell is less than a neighbor cell,
# any out of bounds cells will return true.
# you'll see how this applies down below
grid = gridthings.IntGrid(data, out_of_bounds_value=10)
grid

<IntGrid shape=(5, 10)>

In [3]:
# Access individual cells in the grid with .get
grid.get(1, 1)

IntCell(y=1, x=1, value=9)

In [4]:
# We can read the grid into Pandas easily with grid.values(),
# which is returning a list of lists of cell values (List[List[int]])

df = pandas.DataFrame(grid.values())
df

,0,1,2,3,4,5,6,7,8,9
0,2,1,9,9,9,4,3,2,1,0
1,3,9,8,7,8,9,4,9,2,1
2,9,8,5,6,7,8,9,8,9,2
3,8,7,6,7,8,9,6,7,8,9
4,9,8,9,9,9,6,5,6,7,8


In [5]:
# Alternatively, get a flattened Collection of cells that you
# can iterate through with .flatten()
grid.flatten()[:5]

[IntCell(y=0, x=0, value=2),
 IntCell(y=0, x=1, value=1),
 IntCell(y=0, x=2, value=9),
 IntCell(y=0, x=3, value=9),
 IntCell(y=0, x=4, value=9)]

In [6]:
# Step one, find all the low points in the grid
# grid.peek_linear() returns a Collection of four cells
# from the y/x point you give it.  [left, right, up, down]
#
# According to the Advent of Code page, there should be 4 low points
low_points = []


def is_low_point(y: int, x: int):
    cell = grid.get(y, x)
    for neighbor in grid.peek_linear(y, x):
        # return False if any neighboring cells are smaller
        # it wouldn't be a low point if that's the case
        # remember OutOfBoundCells have value 10 because
        # of the default set earlier on
        if cell > neighbor:
            return False
    return True


for cell in grid.flatten():
    if is_low_point(cell.y, cell.x):
        low_points.append(cell)

low_points

[IntCell(y=0, x=1, value=1),
 IntCell(y=0, x=9, value=0),
 IntCell(y=2, x=2, value=5),
 IntCell(y=4, x=6, value=5)]

In [7]:
# Step two, find basins around those low points
# a basin is all locations neighboring a low point
# that are higher value but not OutOfBounds or value 9
#
# Note that Cells can be compared with >, >=, <, <=
# as if they were plain values, but Cell == Cell
# will compare the dictionaries, so "cell in [cells]"
# still works here
#
# There should be 4 basins, one for each low point
# The sizes should be 3, 9, 14, 9
basins = []


class Basin:
    def __init__(self, start_cell: gridthings.Cell):
        # self.points will be extended during the self.explore
        # method if it finds neighbors matching the right criteria
        # then this for loop will iterate over the new points too
        self.points = [start_cell]
        for point in self.points:
            self.explore(point)

    def explore(self, cell: gridthings.Cell):
        for neighbor in grid.peek_linear(cell.y, cell.x):
            if neighbor not in self.points:
                if neighbor < 9 and neighbor > cell:
                    self.points.append(neighbor)

    def __repr__(self):
        return f"<Basin start={self.points[0]} points={len(self.points)}>"


for point in low_points:
    basin = Basin(start_cell=point)
    basins.append(basin)

basins

[<Basin start=y=0 x=1 value=1 points=3>,
 <Basin start=y=0 x=9 value=0 points=9>,
 <Basin start=y=2 x=2 value=5 points=14>,
 <Basin start=y=4 x=6 value=5 points=9>]

In [8]:
basins[0].points

[IntCell(y=0, x=1, value=1),
 IntCell(y=0, x=0, value=2),
 IntCell(y=1, x=0, value=3)]

In [9]:
basins[1].points

[IntCell(y=0, x=9, value=0),
 IntCell(y=0, x=8, value=1),
 IntCell(y=1, x=9, value=1),
 IntCell(y=0, x=7, value=2),
 IntCell(y=1, x=8, value=2),
 IntCell(y=2, x=9, value=2),
 IntCell(y=0, x=6, value=3),
 IntCell(y=0, x=5, value=4),
 IntCell(y=1, x=6, value=4)]

In [10]:
# Find the three largest Basins and multiply their size together
sorted_basins = list(sorted(basins, key=lambda item: len(item.points)))
sorted_basins

[<Basin start=y=0 x=1 value=1 points=3>,
 <Basin start=y=0 x=9 value=0 points=9>,
 <Basin start=y=4 x=6 value=5 points=9>,
 <Basin start=y=2 x=2 value=5 points=14>]

In [11]:
import math

math.prod([len(basin.points) for basin in sorted_basins[-3:]])

1134